In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold

In [2]:
# train has meter readings, which are obviously lacking in test set
train_df = pd.read_pickle("../data/processed/train_data.pkl")
test_df = pd.read_pickle("../data/processed/test_data.pkl")

In [3]:
train_df.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,weekday,area_per_floor,outlier_square_feet,outlier_area_per_floor,air_temperature_6_lag,air_temperature_24_lag,dew_temperature_6_lag,dew_temperature_24_lag,cloud_coverage_6_lag,cloud_coverage_24_lag
0,0,0,0.000000,0,0,8.913550,11.0,NaN,19.406250,3.162209,...,4,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1142,1,0.000000,13,6,11.549431,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,16.768229,18.847656,11.118490,10.296224,3.326116,3.221908
2,1142,0,48.000000,13,6,11.549431,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,17.519531,18.801758,11.302083,10.154460,3.072498,3.138575
3,1141,0,21.000000,13,8,9.951611,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,18.153646,18.660156,11.402344,9.991862,2.856456,2.971908
4,1140,2,6609.379883,13,0,11.960703,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,18.703125,18.518229,11.302083,9.783366,2.856456,2.721908


In [14]:
train_df.shape

(19229841, 26)

In [24]:
train_df.columns

Index(['building_id', 'meter', 'meter_reading', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'relative_humidity', 'air_temp_f', 'feels_like_temp', 'hour', 'weekday',
       'area_per_floor', 'outlier_square_feet', 'outlier_area_per_floor',
       'air_temperature_6_lag', 'air_temperature_24_lag',
       'dew_temperature_6_lag', 'dew_temperature_24_lag',
       'cloud_coverage_6_lag', 'cloud_coverage_24_lag'],
      dtype='object')

In [4]:
test_df.head()

,row_id,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,weekday,area_per_floor,outlier_square_feet,outlier_area_per_floor,air_temperature_6_lag,air_temperature_24_lag,dew_temperature_6_lag,dew_temperature_24_lag,cloud_coverage_6_lag,cloud_coverage_24_lag
0,0,0,0,0,0,8.913550,11.0,NaN,15.601562,2.0,...,6,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,0,0,0,7.908387,15.0,NaN,15.601562,2.0,...,6,NaN,True,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,0,0,0,8.589700,28.0,NaN,15.601562,2.0,...,6,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,0,0,0,10.072597,17.0,NaN,15.601562,2.0,...,6,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,0,0,0,11.666565,44.0,NaN,15.601562,2.0,...,6,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
test_df.shape

(41697600, 26)

In [23]:
test_df.columns

Index(['row_id', 'building_id', 'meter', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'relative_humidity', 'air_temp_f', 'feels_like_temp', 'hour', 'weekday',
       'area_per_floor', 'outlier_square_feet', 'outlier_area_per_floor',
       'air_temperature_6_lag', 'air_temperature_24_lag',
       'dew_temperature_6_lag', 'dew_temperature_24_lag',
       'cloud_coverage_6_lag', 'cloud_coverage_24_lag'],
      dtype='object')

In [6]:
groups = train_df.building_id

In [28]:
group_kfold = GroupKFold(n_splits = 2)
group_kfold.get_n_splits()

2

In [29]:
group_kfold.split(train_df, train_df["meter_reading"])

<generator object _BaseKFold.split at 0x000002115AD70048>

In [31]:
for train_index, test_index in group_kfold.split(train_df, train_df["meter_reading"], groups):
    print("TRAIN:", train_index, "TEST", test_index)

TRAIN: [       1        2       11 ... 19229835 19229839 19229840] TEST [       0        3        4 ... 19229836 19229837 19229838]
TRAIN: [       0        3        4 ... 19229836 19229837 19229838] TEST [       1        2       11 ... 19229835 19229839 19229840]
